# Train test split KVLCC2 HSVA
<cite id="hzn48"><a href="#zotero|7505983/U2L55C9Q">(Luo et al., 2016)</a></cite>

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
matplotlib.rcParams["figure.figsize"] = (10,10)
from src.symbols import *

# Read configs:
conf_path = os.path.join("../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmm_names = global_variables["vmms"]

from wPCC_pipeline.pipelines.motion_regression.nodes import predict_force, fit_motions, create_model_from_motion_regression
from wPCC_pipeline.pipelines.prediction.nodes import simulate_euler
from src.parameters import df_parameters
from src.models.vmm import manoeuvring model

In [ ]:
ship="kvlcc2_hsva"
#vmm_name = "vmm_martins_simple"
vmm_name = "vmm_abkowitz"
vmm = catalog.load(vmm_name)

ship_data = catalog.load(f"{ship}.ship_data")

train = "HSVA_CPMC_KVLCC2_Z_25_05"
df_train = catalog.load(f'{ship}.updated.{train}.data_ek_smooth')

test = "HSVA_CPMC_KVLCC2_Z_35_05"
df_test = catalog.load(f'{ship}.updated.{test}.data_ek_smooth')

#delta_start = np.deg2rad(10.5)
#
#for data in [df_train, df_test]:
#    index_start = (data['delta'].diff().abs() > 0).idxmax()
#    start = data.loc[0:index_start] 
#    start['delta'] = delta_start

#index_start = (df_train['delta'].diff().abs() > 0).idxmax()
#df_train = df_train.loc[index_start:].copy()
#df_train.index = df_train.index - df_train.index[0]

index_start = (df_test['delta'].diff().abs() > 0).idxmax()
df_test = df_test.loc[index_start:].copy()
df_test.index = df_test.index - df_test.index[0]


added_masses = catalog.load(f"{ship}.added_masses")
exclude_parameters = catalog.load(f"params:{ship}.motion_regression.exclude_parameters")


In [ ]:
data = df_train
data_with_force = predict_force(data=data, added_masses=added_masses, ship_parameters=ship_data, vmm=vmm)


In [ ]:
regression, parameters = fit_motions(data=data_with_force, added_masses=added_masses, ship_data=ship_data, vmm=vmm, exclude_parameters=exclude_parameters)

In [ ]:
mask = regression.parameters['$P_{value}$'] > 10**-3
symbols_ = df_parameters.loc[regression.parameters.loc[mask].index,'symbol']
subs = [(value,0) for value in symbols_.values]
X_eq2 = vmm.X_eq.subs(subs)
Y_eq2 = vmm.Y_eq.subs(subs)
N_eq2 = vmm.N_eq.subs(subs)
vmm2 = manoeuvring model(X_eq=X_eq2, Y_eq=Y_eq2, N_eq=N_eq2)

In [ ]:
X_eq2

In [ ]:
Y_eq2

In [ ]:
N_eq2

In [ ]:
regression2, parameters2 = fit_motions(data=data_with_force, added_masses=added_masses, ship_data=ship_data, vmm=vmm2, exclude_parameters=exclude_parameters)

In [ ]:
regression.parameters['range'] = (regression.parameters['$conf_{higher}$'] - regression.parameters['$conf_{lower}$'])/(regression.parameters['regressed']).abs()
regression.parameters

In [ ]:
len(regression.parameters)

In [ ]:
len(regression2.parameters)

In [ ]:
model2 = create_model_from_motion_regression(regression=regression2)

In [ ]:
parameters3['$P_{value}$'].max()

In [ ]:
X_eq3 = vmm.X_eq
Y_eq3 = vmm.Y_eq
N_eq3 = vmm.N_eq
parameters3 = regression.parameters.copy()
parameters3['range'] = (parameters3['$conf_{higher}$'] - parameters3['$conf_{lower}$'])/(parameters3['regressed']).abs()

In [ ]:
parameters3

In [ ]:
range_critical = 0.8

while parameters3['range'].max() > P_critical:
    
    id = parameters3['range'].idxmax()
    symbol = df_parameters.loc[id]['symbol']
    print(f"removing:{symbol}")
    subs = [(symbol,0)]
    
    X_eq3 = X_eq3.subs(subs)
    Y_eq3 = Y_eq3.subs(subs)
    N_eq3 = N_eq3.subs(subs)
    vmm3 = manoeuvring model(X_eq=X_eq3, Y_eq=Y_eq3, N_eq=N_eq3)
    
    regression3, parameters3 = fit_motions(data=data_with_force, added_masses=added_masses, ship_data=ship_data, vmm=vmm3, exclude_parameters=exclude_parameters)
    parameters3['range'] = (parameters3['$conf_{higher}$'] - parameters3['$conf_{lower}$'])/(parameters3['regressed']).abs()
    
    parameters3.loc['Xu','range'] = 0
    parameters3.loc['Yr','range'] = 0
    parameters3.loc['Yv','range'] = 0
    parameters3.loc['Nr','range'] = 0
    parameters3.loc['Nv','range'] = 0
    parameters3.loc['Ydelta','range'] = 0
    parameters3.loc['Ndelta','range'] = 0
    parameters3.loc['Ythrustdelta','range'] = 0
    parameters3.loc['Nthrustdelta','range'] = 0
    
    
    
    

In [ ]:
len(regression3.parameters)

In [ ]:
regression3.parameters

In [ ]:
display(regression3.X_eq)
display(regression3.Y_eq)
display(regression3.N_eq)

In [ ]:
regression3.model_Y.summary()

In [ ]:
model3 = create_model_from_motion_regression(regression=regression3)

In [ ]:
df_ = df_test.copy()
df_.iloc[0]['r'] = 0
result = model2.simulate(df_=df_)
result3 = model3.simulate(df_=df_)
df_result = result.result.copy()
df_result3 = result3.result.copy()

df_result['psi_deg'] = np.rad2deg(df_result['psi'])
df_result3['psi_deg'] = np.rad2deg(df_result3['psi'])

df_test['psi_deg'] = np.rad2deg(df_test['psi'])
df_result['delta_deg'] = np.rad2deg(df_result['delta'])
df_result3['delta_deg'] = np.rad2deg(df_result3['delta'])

df_test['delta_deg'] = np.rad2deg(df_test['delta'])


In [ ]:
dataframes = {
    'Experiment' : df_test,
    'Prediction' : df_result,
    'Prediction3' : df_result3,
}

styles = {
    'Experiment' : {'style':'r--'},
    'Prediction' : {'style':'b-'},
    'Prediction3' : {'style':'g-'},
}

In [ ]:
track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'],  styles=styles);

In [ ]:
plot(dataframes=dataframes, keys=['delta_deg','psi_deg','y0','u','v','r'], ncols=1, styles=styles, time_window=[0,2]);
plot(dataframes=dataframes, keys=['delta_deg','psi_deg','y0','u','v','r'], ncols=1, styles=styles);

In [ ]:
import statsmodels.api as sm
import seaborn as sns

X = pd.DataFrame()
N = 1000
x1 = X['x1'] = np.linspace(0,1,N)
x2 = X['x2'] = X['x1'] + np.random.normal(scale=0.05, size=N)
y = x1 + x2 + np.random.normal(scale=0.1, size=N)

data = X.copy()
data['y'] = y

sns.pairplot(data=data)

In [ ]:
data.corr()

In [ ]:
model1 = sm.OLS(y,X)
result = model1.fit()
result.summary()

In [ ]:
y2 = y.diff()[1:].copy()
X2 = X.diff().iloc[1:].copy()
model2 = sm.OLS(y2,X2)
result2 = model2.fit()
result2.summary()

In [ ]:
X3 = X[['x1']]
model3 = sm.OLS(y,X3)
result3 = model3.fit()
result3.summary()

In [ ]:
X.corr()

In [ ]:
X2.corr()

In [ ]:
result.condition_number

In [ ]:
result2.condition_number